In [47]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import numpy as np

In [48]:
class Likelihood(nn.Module):
    def __init__(self):
        super().__init__()
        # Parameters for line
        self.m = nn.Parameter(torch.tensor(0.0), requires_grad=True)
        self.b = nn.Parameter(torch.tensor(0.0), requires_grad=True)
        
        # Noise of data
        self.noise = nn.Parameter(torch.tensor(1.0), requires_grad=True)
    
    def forward(self, x, y):
        # Takes numpy array as input and converts to tensors
        x = torch.tensor(x)
        y = torch.tensor(y)
        
        # Minimizing this function
        return (1 / len(x)) * torch.sum((y - x * self.m - self.b) ** 2 / (2 * self.noise ** 2)) + torch.log(self.noise)

In [49]:
class SimpleProb(nn.Module):
    def __init__(self):
        self.std = nn.Parameter(torch.tensor(1.0), requires_grad=True)
    
    def forward(self, k):
        return torch.log(self.std) + 0.5 * (k / self.std) ** 2

In [50]:
class Posterior(nn.Module):
    def __init__(self):
        self.ll = Likelihood()
        self.pm = SimpleProb()
        self.pb = SimpleProb()
    
    def forward(self, x, y):
        return -torch.log(self.ll(x, y) * self.pm(self.ll.m) * self.pb(self.ll.b))

In [51]:
def train(x, y, epochs=5, frac=0.1):
    model = Likelihood()
    optimizer = optim.Adam(model.parameters())
    
    for epoch in tqdm(range(epochs), desc="Training..."):
        # Zero gradiant for training
        optimizer.zero_grad()
        
        # Random selection of data points per iteration
        indices = np.random.randint(low=0, high=len(x), size=int(len(x) * frac))
        
        # Calculates likelihood
        loglik = model(x[indices], y[indices])
        e = torch.mean(loglik)
        
        # Updates parameters
        e.backward()
        optimizer.step()
    
    return model

In [52]:
df = pd.read_csv('data/Advertising.csv').drop('Unnamed: 0', axis=1)
x = df.TV.to_numpy()
y = df.Sales.to_numpy()

In [53]:
model_ll = train(x, y, epochs=5000)
model_ll

Training...: 100%|██████████| 5000/5000 [00:04<00:00, 1045.52it/s]


Likelihood()

In [55]:
(model_ll.m, model_ll.b, model_ll.noise)

(Parameter containing:
 tensor(nan, requires_grad=True),
 Parameter containing:
 tensor(nan, requires_grad=True),
 Parameter containing:
 tensor(nan, requires_grad=True))